In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%capture
!pip install tldextract

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/MOE_DGA/train_wl.csv'
train_df = pd.read_csv(file_path)

# Mezclar las filas aleatoriamente
train_df = train_df.sample(frac=1).reset_index(drop=True)

# Mostrar las primeras filas
train_df


,domain,family,label
0,apadana.skin,legit,notdga
1,viraltecoop.com,legit,notdga
2,korda.co.uk,legit,notdga
3,mesa3.com.br,legit,notdga
4,sport-pack-examination.com,matsnu,dga
...,...,...,...
159995,produccioncientificaluz.org,legit,notdga
159996,dk-zio.ru,legit,notdga
159997,grandcluby4.xyz,legit,notdga
159998,alonetogether.net,suppobox,dga


In [4]:
import pandas as pd
import numpy as np
import re
import math
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import tldextract
from urllib.parse import urlparse

class FANCIFeatureExtractor:
    """
    Extractor de características basado en el paper FANCI
    Implementa las 21 características: 12 estructurales + 7 lingüísticas + 2 estadísticas
    """

    def __init__(self):
        # Conjunto de TLDs válidos (simplificado)
        self.valid_tlds = {
            'com', 'org', 'net', 'edu', 'gov', 'mil', 'int', 'arpa',
            'de', 'uk', 'fr', 'it', 'es', 'ru', 'cn', 'jp', 'br', 'au', 'ca'
        }

        # Conjunto de sufijos públicos comunes
        self.public_suffixes = {
            'com', 'org', 'net', 'edu', 'gov', 'co.uk', 'dyndns.org',
            'ddns.net', 'amazonaws.com'
        }

        # Vocales para características lingüísticas
        self.vowels = set('aeiouAEIOU')
        self.consonants = set('bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ')

    def extract_e2ld(self, domain):
        """Extrae el effective second-level domain (e2LD)"""
        try:
            extracted = tldextract.extract(domain)
            # Si es un servicio DNS dinámico, incluir el subdominio
            if extracted.suffix in ['dyndns.org', 'ddns.net']:
                return f"{extracted.subdomain}.{extracted.domain}" if extracted.subdomain else extracted.domain
            return extracted.domain
        except:
            # Fallback manual
            parts = domain.lower().split('.')
            if len(parts) >= 2:
                return parts[-2]  # Segundo nivel
            return domain

    def get_dot_free_public_suffix_free(self, domain):
        """Obtiene la versión sin puntos y sin sufijo público"""
        e2ld = self.extract_e2ld(domain)
        return re.sub(r'[^a-zA-Z0-9]', '', e2ld)

    def extract_features(self, domain):
        """Extrae las 21 características de FANCI para un dominio"""
        domain = domain.lower().strip()
        e2ld = self.extract_e2ld(domain)
        parts = domain.split('.')
        ddsf = self.get_dot_free_public_suffix_free(domain)

        features = {}

        # =============== CARACTERÍSTICAS ESTRUCTURALES (12) ===============

        # 1. Longitud del dominio
        features['domain_length'] = len(domain)

        # 2. Número de subdominios (excluyendo sufijos públicos)
        features['num_subdomains'] = max(0, len(parts) - 2)  # Simplificado

        # 3. Longitud media de subdominios
        if len(parts) > 1:
            subdomain_lengths = [len(p) for p in parts[:-1]]  # Excluir TLD
            features['subdomain_length_mean'] = np.mean(subdomain_lengths) if subdomain_lengths else 0
        else:
            features['subdomain_length_mean'] = len(e2ld)

        # 4. Tiene prefijo www
        features['has_www_prefix'] = 1 if domain.startswith('www.') else 0

        # 5. Tiene TLD válido
        tld = parts[-1] if len(parts) > 1 else ''
        features['has_valid_tld'] = 1 if tld in self.valid_tlds else 0

        # 6. Contiene subdominio de un solo carácter
        features['contains_single_char_subdomain'] = 1 if any(len(p) == 1 for p in parts[:-1]) else 0

        # 7. Es repetición exclusiva de prefijo
        features['is_exclusive_prefix_repetition'] = self._is_prefix_repetition(domain)

        # 8. Contiene TLD como subdominio
        features['contains_tld_as_subdomain'] = 1 if any(p in self.valid_tlds for p in parts[:-1]) else 0

        # 9. Ratio de subdominios que son solo dígitos
        digit_subdomains = sum(1 for p in parts[:-1] if p.isdigit())
        total_subdomains = max(1, len(parts) - 1)
        features['ratio_digit_exclusive_subdomains'] = digit_subdomains / total_subdomains

        # 10. Ratio de subdominios hexadecimales
        hex_subdomains = sum(1 for p in parts[:-1] if self._is_hex(p))
        features['ratio_hex_exclusive_subdomains'] = hex_subdomains / total_subdomains

        # 11. Ratio de guiones bajos
        features['underscore_ratio'] = ddsf.count('_') / max(1, len(ddsf))

        # 12. Contiene dirección IP
        features['contains_ip_address'] = 1 if self._contains_ip(domain) else 0

        # =============== CARACTERÍSTICAS LINGÜÍSTICAS (7) ===============

        # 13. Contiene dígitos
        features['contains_digits'] = 1 if any(c.isdigit() for c in ddsf) else 0

        # 14. Ratio de vocales
        vowel_count = sum(1 for c in ddsf if c in self.vowels)
        features['vowel_ratio'] = vowel_count / max(1, len(ddsf))

        # 15. Ratio de dígitos
        digit_count = sum(1 for c in ddsf if c.isdigit())
        features['digit_ratio'] = digit_count / max(1, len(ddsf))

        # 16. Cardinalidad del alfabeto
        features['alphabet_cardinality'] = len(set(ddsf))

        # 17. Ratio de caracteres repetidos
        char_counts = Counter(ddsf)
        repeated_chars = sum(1 for count in char_counts.values() if count > 1)
        features['ratio_repeated_characters'] = repeated_chars / max(1, len(set(ddsf)))

        # 18. Ratio de consonantes consecutivas
        features['ratio_consecutive_consonants'] = self._consecutive_ratio(ddsf, self.consonants)

        # 19. Ratio de dígitos consecutivos
        features['ratio_consecutive_digits'] = self._consecutive_digits_ratio(ddsf)

        # =============== CARACTERÍSTICAS ESTADÍSTICAS (2) ===============

        # 20. Distribución de n-gramas (simplificado a 1-grama)
        features.update(self._ngram_features(ddsf))

        # 21. Entropía
        features['entropy'] = self._calculate_entropy(ddsf)

        return features

    def _is_prefix_repetition(self, domain):
        """Verifica si el dominio es una repetición de prefijo exclusiva"""
        parts = domain.split('.')
        if len(parts) < 2:
            return 0

        base = parts[0]
        for i in range(1, len(parts)-1):  # Excluir TLD
            if not parts[i].startswith(base):
                return 0
        return 1

    def _is_hex(self, s):
        """Verifica si una cadena es hexadecimal"""
        try:
            int(s, 16)
            return len(s) > 0
        except ValueError:
            return False

    def _contains_ip(self, domain):
        """Verifica si contiene una dirección IP"""
        ip_pattern = r'\b(?:\d{1,3}\.){3}\d{1,3}\b'
        return bool(re.search(ip_pattern, domain))

    def _consecutive_ratio(self, text, char_set):
        """Calcula el ratio de secuencias consecutivas de caracteres del conjunto"""
        if not text:
            return 0

        consecutive_lengths = []
        current_length = 0

        for char in text:
            if char in char_set:
                current_length += 1
            else:
                if current_length >= 2:
                    consecutive_lengths.append(current_length)
                current_length = 0

        if current_length >= 2:
            consecutive_lengths.append(current_length)

        return sum(consecutive_lengths) / max(1, len(text))

    def _consecutive_digits_ratio(self, text):
        """Calcula el ratio de dígitos consecutivos"""
        if not text:
            return 0

        consecutive_lengths = []
        current_length = 0

        for char in text:
            if char.isdigit():
                current_length += 1
            else:
                if current_length >= 2:
                    consecutive_lengths.append(current_length)
                current_length = 0

        if current_length >= 2:
            consecutive_lengths.append(current_length)

        return sum(consecutive_lengths) / max(1, len(text))

    def _ngram_features(self, text, n=1):
        """Extrae características de n-gramas (simplificado)"""
        if not text:
            return {
                'ngram_mean': 0,
                'ngram_std': 0,
                'ngram_min': 0,
                'ngram_max': 0,
                'ngram_median': 0,
                'ngram_q25': 0,
                'ngram_q75': 0
            }

        # Frecuencias de 1-gramas
        char_freq = Counter(text)
        frequencies = list(char_freq.values())

        if not frequencies:
            return {
                'ngram_mean': 0,
                'ngram_std': 0,
                'ngram_min': 0,
                'ngram_max': 0,
                'ngram_median': 0,
                'ngram_q25': 0,
                'ngram_q75': 0
            }

        return {
            'ngram_mean': np.mean(frequencies),
            'ngram_std': np.std(frequencies),
            'ngram_min': np.min(frequencies),
            'ngram_max': np.max(frequencies),
            'ngram_median': np.median(frequencies),
            'ngram_q25': np.percentile(frequencies, 25),
            'ngram_q75': np.percentile(frequencies, 75)
        }

    def _calculate_entropy(self, text):
        """Calcula la entropía de Shannon"""
        if not text:
            return 0

        char_freq = Counter(text)
        text_len = len(text)
        entropy = 0

        for count in char_freq.values():
            p = count / text_len
            if p > 0:
                entropy -= p * math.log2(p)

        return entropy

class FANCIRandomForest:
    """
    Implementación del Random Forest de FANCI
    """

    def __init__(self):
        # Parámetros optimizados según el paper
        self.rf = RandomForestClassifier(
            n_estimators=785,        # T parameter
            max_features=18,         # F parameter
            criterion='gini',        # i(N) parameter
            random_state=42,
            n_jobs=-1
        )
        self.feature_extractor = FANCIFeatureExtractor()
        self.feature_names = None
        self.is_trained = False

    def extract_features_from_dataframe(self, df):
        """Extrae características de un DataFrame"""
        print("Extrayendo características de dominios...")

        feature_list = []
        for idx, domain in enumerate(df['domain']):
            if idx % 10000 == 0:
                print(f"Procesando dominio {idx}/{len(df)}")

            features = self.feature_extractor.extract_features(domain)
            feature_list.append(features)

        # Convertir a DataFrame
        feature_df = pd.DataFrame(feature_list)

        # Asegurar orden consistente de características
        if self.feature_names is None:
            # Primera vez - guardar el orden
            self.feature_names = sorted(feature_df.columns.tolist())

        # Reordenar columnas según el orden establecido
        feature_df = feature_df.reindex(columns=self.feature_names, fill_value=0)

        return feature_df

    def prepare_labels(self, df):
        """Prepara las etiquetas (0=benigno, 1=malicioso)"""
        # Convertir 'dga' -> 1, 'notdga' -> 0
        return (df['label'] == 'dga').astype(int)

    def train(self, train_df):
        """Entrena el modelo con el DataFrame"""
        print("=== ENTRENANDO FANCI RANDOM FOREST ===")

        # Extraer características
        X = self.extract_features_from_dataframe(train_df)
        y = self.prepare_labels(train_df)

        print(f"Dimensiones del dataset: {X.shape}")
        print(f"Distribución de clases: {Counter(y)}")
        print(f"Características: {self.feature_names}")

        # Entrenar modelo
        print("Entrenando Random Forest...")
        self.rf.fit(X, y)
        self.is_trained = True

        # Evaluación con validación cruzada
        print("Evaluando con 5-fold cross-validation...")
        cv_scores = cross_val_score(
            self.rf, X, y,
            cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
            scoring='accuracy'
        )

        print(f"Accuracy promedio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
        print(f"Scores individuales: {cv_scores}")

        return self

    def predict(self, domains):
        """Predice si los dominios son DGA o no"""
        if not self.is_trained:
            raise ValueError("El modelo debe ser entrenado antes de hacer predicciones")

        if isinstance(domains, str):
            domains = [domains]

        # Crear DataFrame temporal
        temp_df = pd.DataFrame({'domain': domains})
        X = self.extract_features_from_dataframe(temp_df)

        predictions = self.rf.predict(X)
        probabilities = self.rf.predict_proba(X)

        results = []
        for i, domain in enumerate(domains):
            results.append({
                'domain': domain,
                'prediction': 'DGA' if predictions[i] == 1 else 'Benign',
                'dga_probability': probabilities[i][1],
                'confidence': max(probabilities[i])
            })

        return results

    def get_feature_importance(self, top_n=10):
        """Obtiene la importancia de las características"""
        if not self.is_trained or self.feature_names is None:
            return None

        importance_df = pd.DataFrame({
            'feature': self.feature_names,
            'importance': self.rf.feature_importances_
        }).sort_values('importance', ascending=False)

        return importance_df.head(top_n)

# =============== EJEMPLO DE USO ===============

def train_fanci_model(train_df):
    """
    Función principal para entrenar el modelo FANCI
    """
    # Crear y entrenar modelo
    fanci_model = FANCIRandomForest()
    fanci_model.train(train_df)

    # Mostrar características más importantes
    print("\n=== CARACTERÍSTICAS MÁS IMPORTANTES ===")
    importance = fanci_model.get_feature_importance()
    print(importance)

    return fanci_model

# Ejemplo de uso con tu DataFrame
if __name__ == "__main__":
    # Supongamos que tienes tu train_df cargado
    # train_df = pd.read_csv('your_data.csv')

    # Entrenar modelo
    #model = train_fanci_model(train_df)

    # Ejemplo de predicción
    test_domains = [
        'google.com',
        'asjdkajsdkjasd.com',
        'facebook.com',
        'xjkdh47dhgkjdf.net'
    ]

    #predictions = model.predict(test_domains)

    #print("\n=== PREDICCIONES DE EJEMPLO ===")
    #for pred in predictions:
        #print(f"Dominio: {pred['domain']}")
        #print(f"Predicción: {pred['prediction']}")
        #print(f"Probabilidad DGA: {pred['dga_probability']:.3f}")
        #print(f"Confianza: {pred['confidence']:.3f}")
        #print("-" * 40)

In [ ]:
import pickle
import joblib
import json
import os
from datetime import datetime

# =============== MÉTODO 1: GUARDAR CON PICKLE ===============

def save_fanci_model_pickle(model, filename="fanci_model.pkl"):
    """
    Guarda el modelo completo usando pickle
    """
    # En Google Colab, guardar en /content/ que está montado en Drive
    filepath = f"/content/{filename}"

    try:
        with open(filepath, 'wb') as f:
            pickle.dump(model, f)
        print(f"✅ Modelo guardado exitosamente en: {filepath}")
        print(f"📁 Tamaño del archivo: {os.path.getsize(filepath) / (1024*1024):.2f} MB")
        return filepath
    except Exception as e:
        print(f"❌ Error al guardar: {e}")
        return None

def load_fanci_model_pickle(filename="fanci_model.pkl"):
    """
    Carga el modelo desde pickle
    """
    filepath = f"/content/{filename}"

    try:
        with open(filepath, 'rb') as f:
            model = pickle.load(f)
        print(f"✅ Modelo cargado exitosamente desde: {filepath}")
        return model
    except Exception as e:
        print(f"❌ Error al cargar: {e}")
        return None

# =============== MÉTODO 2: GUARDAR CON JOBLIB (RECOMENDADO) ===============

def save_fanci_model_joblib(model, filename="fanci_model.joblib"):
    """
    Guarda el modelo usando joblib (más eficiente para sklearn)
    """
    filepath = f"/content/{filename}"

    try:
        joblib.dump(model, filepath, compress=3)  # compress=3 para reducir tamaño
        print(f"✅ Modelo guardado exitosamente en: {filepath}")
        print(f"📁 Tamaño del archivo: {os.path.getsize(filepath) / (1024*1024):.2f} MB")
        return filepath
    except Exception as e:
        print(f"❌ Error al guardar: {e}")
        return None

def load_fanci_model_joblib(filename="fanci_model.joblib"):
    """
    Carga el modelo desde joblib
    """
    filepath = f"/content/{filename}"

    try:
        model = joblib.load(filepath)
        print(f"✅ Modelo cargado exitosamente desde: {filepath}")
        return model
    except Exception as e:
        print(f"❌ Error al cargar: {e}")
        return None

# =============== MÉTODO 3: GUARDAR CON METADATOS ===============

def save_fanci_model_complete(model, base_filename="fanci_model"):
    """
    Guarda el modelo completo con metadatos
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Archivos a crear
    model_file = f"/content/{base_filename}_{timestamp}.joblib"
    metadata_file = f"/content/{base_filename}_{timestamp}_metadata.json"

    try:
        # 1. Guardar el modelo
        joblib.dump(model, model_file, compress=3)

        # 2. Guardar metadatos
        metadata = {
            "timestamp": timestamp,
            "model_type": "FANCI Random Forest",
            "sklearn_version": "1.3+",
            "feature_names": model.feature_names if hasattr(model, 'feature_names') else None,
            "is_trained": model.is_trained if hasattr(model, 'is_trained') else False,
            "rf_params": {
                "n_estimators": model.rf.n_estimators,
                "max_features": model.rf.max_features,
                "criterion": model.rf.criterion,
                "random_state": model.rf.random_state
            } if hasattr(model, 'rf') else None,
            "file_size_mb": round(os.path.getsize(model_file) / (1024*1024), 2)
        }

        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)

        print(f"✅ Modelo guardado en: {model_file}")
        print(f"✅ Metadatos guardados en: {metadata_file}")
        print(f"📁 Tamaño total: {metadata['file_size_mb']} MB")

        return model_file, metadata_file

    except Exception as e:
        print(f"❌ Error al guardar: {e}")
        return None, None

def load_fanci_model_complete(base_filename="fanci_model", timestamp=None):
    """
    Carga el modelo con metadatos
    """
    if timestamp is None:
        # Buscar el archivo más reciente
        files = [f for f in os.listdir("/content/") if f.startswith(base_filename) and f.endswith(".joblib")]
        if not files:
            print("❌ No se encontraron archivos del modelo")
            return None, None

        # Ordenar por timestamp y tomar el más reciente
        files.sort(reverse=True)
        model_file = f"/content/{files[0]}"
        timestamp = files[0].split('_')[-1].replace('.joblib', '')
    else:
        model_file = f"/content/{base_filename}_{timestamp}.joblib"

    metadata_file = f"/content/{base_filename}_{timestamp}_metadata.json"

    try:
        # Cargar modelo
        model = joblib.load(model_file)

        # Cargar metadatos
        metadata = None
        if os.path.exists(metadata_file):
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)

        print(f"✅ Modelo cargado desde: {model_file}")
        if metadata:
            print(f"📊 Timestamp: {metadata['timestamp']}")
            print(f"📊 Tamaño: {metadata['file_size_mb']} MB")

        return model, metadata

    except Exception as e:
        print(f"❌ Error al cargar: {e}")
        return None, None

# =============== MÉTODO 4: MONTAR GOOGLE DRIVE ===============

def mount_google_drive():
    """
    Monta Google Drive en Colab
    """
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive montado exitosamente")
        return True
    except ImportError:
        print("❌ Este código debe ejecutarse en Google Colab")
        return False
    except Exception as e:
        print(f"❌ Error al montar Drive: {e}")
        return False

def save_to_drive(model, folder_path="MyDrive/ML_Models", filename="fanci_model"):
    """
    Guarda el modelo directamente en una carpeta de Google Drive
    """
    # Montar Drive si no está montado
    if not os.path.exists('/content/drive'):
        if not mount_google_drive():
            return None

    # Crear carpeta si no existe
    full_folder_path = f"/content/drive/{folder_path}"
    os.makedirs(full_folder_path, exist_ok=True)

    # Generar nombre con timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_path = f"{full_folder_path}/{filename}_{timestamp}.joblib"
    metadata_path = f"{full_folder_path}/{filename}_{timestamp}_metadata.json"

    try:
        # Guardar modelo
        joblib.dump(model, model_path, compress=3)

        # Guardar metadatos
        metadata = {
            "timestamp": timestamp,
            "model_type": "FANCI Random Forest",
            "drive_path": model_path,
            "accuracy": "73%",  # Puedes actualizar esto con el accuracy real
            "feature_count": len(model.feature_names) if hasattr(model, 'feature_names') else None,
            "file_size_mb": round(os.path.getsize(model_path) / (1024*1024), 2)
        }

        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)

        print(f"✅ Modelo guardado en Google Drive:")
        print(f"📁 Ubicación: {model_path}")
        print(f"📊 Tamaño: {metadata['file_size_mb']} MB")

        return model_path

    except Exception as e:
        print(f"❌ Error al guardar en Drive: {e}")
        return None

# =============== FUNCIONES DE UTILIDAD ===============

def list_saved_models(folder_path="MyDrive/ML_Models"):
    """
    Lista todos los modelos guardados en Drive
    """
    full_folder_path = f"/content/drive/{folder_path}"

    if not os.path.exists(full_folder_path):
        print(f"❌ La carpeta {folder_path} no existe")
        return []

    model_files = [f for f in os.listdir(full_folder_path) if f.endswith('.joblib')]

    print(f"📁 Modelos encontrados en {folder_path}:")
    for i, file in enumerate(model_files, 1):
        file_path = os.path.join(full_folder_path, file)
        size_mb = os.path.getsize(file_path) / (1024*1024)
        print(f"{i}. {file} ({size_mb:.2f} MB)")

    return model_files

def load_from_drive(filename, folder_path="MyDrive/ML_Models"):
    """
    Carga un modelo específico desde Google Drive
    """
    full_path = f"/content/drive/{folder_path}/{filename}"

    try:
        model = joblib.load(full_path)
        print(f"✅ Modelo cargado desde Drive: {full_path}")
        return model
    except Exception as e:
        print(f"❌ Error al cargar desde Drive: {e}")
        return None

# =============== EJEMPLO DE USO COMPLETO ===============

def example_usage():
    """
    Ejemplo de cómo usar las funciones de guardado
    """
    # Supongamos que tienes tu modelo entrenado
    # model = tu_modelo_entrenado

    print("=== OPCIONES DE GUARDADO ===")
    print("1. Guardar localmente con joblib (recomendado)")
    print("2. Guardar con metadatos completos")
    print("3. Guardar directamente en Google Drive")

    # Opción 1: Guardado simple
    # save_fanci_model_joblib(model)

    # Opción 2: Guardado con metadatos
    # save_fanci_model_complete(model)

    # Opción 3: Guardado en Drive
    # save_to_drive(model)

    print("\n=== OPCIONES DE CARGA ===")
    print("1. Cargar desde archivo local")
    print("2. Cargar desde Google Drive")
    print("3. Listar modelos disponibles")

    # Ejemplos de carga
    # model = load_fanci_model_joblib()
    # model = load_from_drive("fanci_model_20241220_143052.joblib")
    # list_saved_models()

# =============== CÓDIGO PARA USAR DIRECTAMENTE ===============

"""
# GUARDAR TU MODELO (elige una opción):

# Opción 1 - Simple y rápido:
save_fanci_model_joblib(model, "mi_fanci_model.joblib")

# Opción 2 - Con metadatos:
save_fanci_model_complete(model, "mi_fanci_model")

# Opción 3 - Directamente en Google Drive:
mount_google_drive()  # Solo si no está montado
save_to_drive(model, "MyDrive/Modelos_ML", "fanci_dga_detector")

# CARGAR TU MODELO:

# Opción 1 - Desde archivo local:
loaded_model = load_fanci_model_joblib("mi_fanci_model.joblib")

# Opción 2 - Desde Google Drive:
loaded_model = load_from_drive("fanci_dga_detector_20241220_143052.joblib", "MyDrive/Modelos_ML")

# VERIFICAR QUE FUNCIONA:
if loaded_model:
    predictions = loaded_model.predict(['google.com', 'maliciousdomain123.com'])
    print("Predicciones:", predictions)
"""

'\n# GUARDAR TU MODELO (elige una opción):\n\n# Opción 1 - Simple y rápido:\nsave_fanci_model_joblib(model, "mi_fanci_model.joblib")\n\n# Opción 2 - Con metadatos:\nsave_fanci_model_complete(model, "mi_fanci_model")\n\n# Opción 3 - Directamente en Google Drive:\nmount_google_drive()  # Solo si no está montado\nsave_to_drive(model, "MyDrive/Modelos_ML", "fanci_dga_detector")\n\n# CARGAR TU MODELO:\n\n# Opción 1 - Desde archivo local:\nloaded_model = load_fanci_model_joblib("mi_fanci_model.joblib")\n\n# Opción 2 - Desde Google Drive:\nloaded_model = load_from_drive("fanci_dga_detector_20241220_143052.joblib", "MyDrive/Modelos_ML")\n\n# VERIFICAR QUE FUNCIONA:\nif loaded_model:\n    predictions = loaded_model.predict([\'google.com\', \'maliciousdomain123.com\'])\n    print("Predicciones:", predictions)\n'

In [ ]:
save_fanci_model_complete(model, "mi_fanci_model")


✅ Modelo guardado en: /content/mi_fanci_model_20250618_164352.joblib
✅ Metadatos guardados en: /content/mi_fanci_model_20250618_164352_metadata.json
📁 Tamaño total: 396.6 MB


('/content/mi_fanci_model_20250618_164352.joblib',
 '/content/mi_fanci_model_20250618_164352_metadata.json')

In [ ]:
mount_google_drive()  # Solo si no está montado
save_to_drive(model, "MyDrive/Modelos_ML", "fanci_dga_detector")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive montado exitosamente
✅ Modelo guardado en Google Drive:
📁 Ubicación: /content/drive/MyDrive/Modelos_ML/fanci_dga_detector_20250618_164818.joblib
📊 Tamaño: 396.6 MB


'/content/drive/MyDrive/Modelos_ML/fanci_dga_detector_20250618_164818.joblib'

In [ ]:
import pandas as pd
import time

# =============== CARGAR MODELO FANCI JOBLIB ===============


from joblib import load

def load_fanci_model():
    """
    Carga el modelo FANCI desde archivo joblib en Google Drive
    """
    try:
        print("🔄 Cargando modelo FANCI...")
        ruta_modelo = "/content/drive/MyDrive/Modelos_ML/fanci_dga_detector_20250618_164818.joblib"
        model = load(ruta_modelo)
        print("✅ Modelo FANCI cargado exitosamente")
        return model
    except Exception as e:
        print(f"❌ Error cargando modelo FANCI: {e}")
        return None


# =============== EVALUACIÓN SIMPLE ===============

families = [
    'matsnu.gz',
    'suppobox.gz',
    'charbot.gz',
    'gozi.gz',
    'manuelita.gz',
    'rovnix.gz',
    'deception.gz',
    'nymaim.gz'
]

runs = 30

# Cargar modelo FANCI una sola vez
fanci_model = load_fanci_model()

if fanci_model is None:
    print("❌ No se pudo cargar el modelo FANCI. Deteniendo evaluación.")
else:
    print("🚀 Iniciando evaluación del modelo FANCI...")

    for family in families:
        print(f"📂 Evaluando familia: {family}")

        # Cargar datos
        dga = pd.read_csv(f'/content/drive/My Drive/Familias_Test/{family}', chunksize=50)
        legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)

        for run in range(runs):
            print(f'  🔄 {run:2}/{runs}', end='\r')

            # Obtener chunks
            dfw = pd.concat([dga.get_chunk(), legit.get_chunk()])

            # Listas para resultados
            pred = []
            prob = []
            query_time = []

            # Evaluar cada dominio
            for domain_to_check in dfw.domain.values:
                st = time.time()

                # =============== PREDICCIÓN CON FANCI ===============
                try:
                    fanci_results = fanci_model.predict([domain_to_check])
                    result = fanci_results[0]

                    label_value = 1 if result['prediction'] == 'DGA' else 0
                    probability = result['dga_probability']

                    pred.append(label_value)
                    prob.append(probability)

                except Exception as e:
                    # Valor por defecto en caso de error
                    pred.append(0)
                    prob.append(0.5)

                query_time.append(time.time() - st)

            # Agregar resultados al DataFrame
            dfw['pred'] = pred
            dfw['prob'] = prob
            dfw['query_time'] = query_time

            # Guardar resultados
            dfw.to_csv(f'/content/drive/My Drive/results/results_FANCI_{family}_{run}.csv.gz', index=False)

    print("\n✅ Evaluación completada!")

Streaming output truncated to the last 5000 lines.
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios.

In [13]:
import pandas as pd
import time

# =============== CARGAR MODELO FANCI JOBLIB ===============


from joblib import load

def load_fanci_model():
    """
    Carga el modelo FANCI desde archivo joblib en Google Drive
    """
    try:
        print("🔄 Cargando modelo FANCI...")
        ruta_modelo = "/content/drive/MyDrive/Modelos_ML/fanci_dga_detector_20250618_164818.joblib"
        model = load(ruta_modelo)
        print("✅ Modelo FANCI cargado exitosamente")
        return model
    except Exception as e:
        print(f"❌ Error cargando modelo FANCI: {e}")
        return None


# =============== EVALUACIÓN SIMPLE ===============

# ✅ Familias que quieres evaluar
families = ['bigviktor.gz',
            'pizd.gz',
            'ngioweb.gz'


           ]

runs = 30

# Cargar modelo FANCI una sola vez
fanci_model = load_fanci_model()

if fanci_model is None:
    print("❌ No se pudo cargar el modelo FANCI. Deteniendo evaluación.")
else:
    print("🚀 Iniciando evaluación del modelo FANCI...")

    for family in families:
        print(f"📂 Evaluando familia: {family}")

        # Cargar datos
        dga = pd.read_csv(f'/content/drive/My Drive/New_Families/{family}', chunksize=50)
        legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)

        for run in range(runs):
            print(f'  🔄 {run:2}/{runs}', end='\r')

            # Obtener chunks
            dfw = pd.concat([dga.get_chunk(), legit.get_chunk()])

            # Listas para resultados
            pred = []
            prob = []
            query_time = []

            # Evaluar cada dominio
            for domain_to_check in dfw.domain.values:
                st = time.time()

                # =============== PREDICCIÓN CON FANCI ===============
                try:
                    fanci_results = fanci_model.predict([domain_to_check])
                    result = fanci_results[0]

                    label_value = 1 if result['prediction'] == 'DGA' else 0
                    probability = result['dga_probability']

                    pred.append(label_value)
                    prob.append(probability)

                except Exception as e:
                    # Valor por defecto en caso de error
                    pred.append(0)
                    prob.append(0.5)

                query_time.append(time.time() - st)

            # Agregar resultados al DataFrame
            dfw['pred'] = pred
            dfw['prob'] = prob
            dfw['query_time'] = query_time

            # Guardar resultados
            dfw.to_csv(f'/content/drive/My Drive/results/results_FANCI_{family}_{run}.csv.gz', index=False)

    print("\n✅ Evaluación completada!")

Streaming output truncated to the last 5000 lines.
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios...
Procesando dominio 0/1
Extrayendo características de dominios.

In [14]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix
)
import matplotlib.pyplot as plt
import seaborn as sns

# Montar Google Drive (si aún no está montado)
from google.colab import drive
drive.mount('/content/drive')

# Función para calcular FPR y TPR
def fpr_tpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    fpr = fp / (fp + tn + 1e-10)  # Evita división por cero
    tpr = tp / (tp + fn + 1e-10)
    return fpr, tpr

# Lista de familias de malware
families = [
    'matsnu.gz',
    'suppobox.gz',
    'charbot.gz',
    'gozi.gz',
    'manuelita.gz',
    'rovnix.gz',
    'deception.gz',
    'nymaim.gz',
    'bigviktor.gz',
    'pizd.gz',
    'ngioweb.gz'
]
runs = 30

# Listas para métricas globales
all_acc, all_pre, all_rec, all_f1 = [], [], [], []
all_fpr, all_tpr, all_qt, all_qts = [], [], [], []
total_unknowns_global = 0

for family in families:
    acc, pre, rec, f1, fpr, tpr, qt, qts = [], [], [], [], [], [], [], []
    total_unknowns = 0

    for run in range(runs):
        path = f'/content/drive/My Drive/results/results_FANCI_{family}_{run}.csv.gz'
        if not os.path.exists(path):
            print(f"[⚠️] Archivo no encontrado: {path}")
            continue

        df = pd.read_csv(path)

        # Reemplazar -1 por 1 en 'pred', si aplica

        y_true = (df['label'] == 'dga').astype(int)

        y_pred = df['pred']

        # Métricas
        acc.append(accuracy_score(y_true, y_pred))
        pre.append(precision_score(y_true, y_pred, zero_division=0))
        rec.append(recall_score(y_true, y_pred, zero_division=0))
        f1.append(f1_score(y_true, y_pred, zero_division=0))
        fpr_val, tpr_val = fpr_tpr(y_true, y_pred)
        fpr.append(fpr_val)
        tpr.append(tpr_val)

        if 'query_time' in df.columns:
            qt.append(df['query_time'].mean())
            qts.append(df['query_time'].std())

    # Promedios por familia
    if acc:  # solo si hubo archivos válidos
        print(f'{family.split(".")[0]:15}: '
              f'acc:{np.mean(acc):.2f}±{np.std(acc):.3f} '
              f'f1:{np.mean(f1):.2f}±{np.std(f1):.3f} '
              f'pre:{np.mean(pre):.2f}±{np.std(pre):.3f} '
              f'rec:{np.mean(rec):.2f}±{np.std(rec):.3f} '
              f'FPR:{np.mean(fpr):.2f}±{np.std(fpr):.3f} '
              f'TPR:{np.mean(tpr):.2f}±{np.std(tpr):.3f} '
              f'QT:{np.mean(qt):.5f}±{np.std(qt):.5f} '
              f'Unknowns: {total_unknowns}')

        all_acc.append(np.mean(acc))
        all_pre.append(np.mean(pre))
        all_rec.append(np.mean(rec))
        all_f1.append(np.mean(f1))
        all_fpr.append(np.mean(fpr))
        all_tpr.append(np.mean(tpr))
        all_qt.append(np.mean(qt))
        all_qts.append(np.mean(qts))
        total_unknowns_global += total_unknowns

# 🔍 Métricas globales
print("\n### 📊 Métricas globales ###")
print(f'Accuracy   : {np.mean(all_acc):.2f}')
print(f'F1-Score   : {np.mean(all_f1):.2f}')
print(f'Precision  : {np.mean(all_pre):.2f}')
print(f'Recall     : {np.mean(all_rec):.2f}')
print(f'FPR        : {np.mean(all_fpr):.2f}')
print(f'TPR        : {np.mean(all_tpr):.2f}')
print(f'Query time : {np.mean(all_qt):.5f} ± {np.mean(all_qts):.5f}')
print(f'Total unknown classifications: {total_unknowns_global}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
matsnu         : acc:0.81±0.038 f1:0.83±0.034 pre:0.77±0.039 rec:0.90±0.040 FPR:0.28±0.055 TPR:0.90±0.040 QT:0.32510±0.00939 Unknowns: 0
suppobox       : acc:0.73±0.055 f1:0.74±0.060 pre:0.73±0.048 rec:0.74±0.083 FPR:0.28±0.055 TPR:0.74±0.083 QT:0.30926±0.00550 Unknowns: 0
charbot        : acc:0.66±0.051 f1:0.64±0.060 pre:0.69±0.058 rec:0.60±0.069 FPR:0.28±0.055 TPR:0.60±0.069 QT:0.31047±0.00618 Unknowns: 0
gozi           : acc:0.74±0.060 f1:0.74±0.065 pre:0.73±0.051 rec:0.75±0.093 FPR:0.28±0.055 TPR:0.75±0.093 QT:0.31124±0.00603 Unknowns: 0
manuelita      : acc:0.47±0.034 f1:0.29±0.056 pre:0.44±0.072 rec:0.22±0.050 FPR:0.28±0.055 TPR:0.22±0.050 QT:0.30536±0.00557 Unknowns: 0
rovnix         : acc:0.84±0.034 f1:0.85±0.028 pre:0.78±0.037 rec:0.95±0.028 FPR:0.28±0.055 TPR:0.95±0.028 QT:0.30645±0.00446 Unknowns: 0
deception      : acc:0.86±0.028 f1:0.88±0.022 pre

In [5]:
path = f'/content/drive/My Drive/results/results_FANCI_matsnu.gz_1.csv.gz'
df = pd.read_csv(path)
df.head(5)

,Unnamed: 0,domain,family,subfamily,label,pred,prob,query_time
0,1048356,shot-devil-camp.com,matsnu,matsnu,dga,1,0.821930,0.341846
1,1048357,tree-candle-trust.com,matsnu,matsnu,dga,1,0.998726,0.312851
2,1048358,camp-taste-grandfather.com,matsnu,matsnu,dga,1,0.650997,0.326123
3,1048359,zoneparent-card.com,matsnu,matsnu,dga,1,0.786234,0.283441
4,1048360,devilsort-self.com,matsnu,matsnu,dga,1,0.767620,0.287343
